In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install ultralytics -q
!pip install opencv-python -q
!pip install --upgrade ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.6/911.6 kB 57.8 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/chequanghuy/Character-Time-series-Matching.git

Cloning into 'Character-Time-series-Matching'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 231 (delta 48), reused 221 (delta 46), pack-reused 7 (from 1)
Receiving objects: 100% (231/231), 21.61 MiB | 12.07 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [ ]:
import cv2
import os

def video_to_frames(video_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Rotate the frame by 180 degrees
        rotated_frame = cv2.rotate(frame, cv2.ROTATE_180)
        # Save the rotated frame
        output_path = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(output_path, frame)

        frame_count += 1

    cap.release()
    print(f"Extracted and rotated {frame_count} frames to {output_dir}")

if __name__ == "__main__":
    video_path = "/content/gdrive/MyDrive/GX010085.MP4"
    output_dir = "/content/Character-Time-series-Matching/Vietnamese/Vietnamese_imgs"
    video_to_frames(video_path, output_dir)

KeyboardInterrupt: 

In [ ]:
from ultralytics import YOLO
start_frame = 0
end_frame = 2664
step = 12
model = YOLO("yolov8m.pt")

for i in range(start_frame, end_frame + 1, step):
    frame_name = f"frame_{i:04d}.jpg"
    print(f"Generating prediction for {frame_name}...")
    model.predict(source=f"/content/{frame_name}", save=True, conf=0.2, iou=0.5)
    # MODEL.predict(f"/content/out/{frame_name}", save=True, show_labels=False)   # carrying out inference

Generating prediction for frame_0000.jpg...

image 1/1 /content/frame_0000.jpg: 384x640 5 cars, 1 bus, 82.5ms
Speed: 13.4ms preprocess, 82.5ms inference, 806.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
Generating prediction for frame_0012.jpg...

image 1/1 /content/frame_0012.jpg: 384x640 4 cars, 1 bus, 9.9ms
Speed: 2.3ms preprocess, 9.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
Generating prediction for frame_0024.jpg...

image 1/1 /content/frame_0024.jpg: 384x640 4 cars, 1 bus, 9.9ms
Speed: 2.4ms preprocess, 9.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
Generating prediction for frame_0036.jpg...

image 1/1 /content/frame_0036.jpg: 384x640 4 cars, 1 bus, 9.8ms
Speed: 2.2ms preprocess, 9.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
Generating prediction for f

In [ ]:
import json
import os
import glob
from ultralytics import YOLO
from PIL import Image
import cv2
from DETECTION import Detection  # Assuming your Detection class is in Detection.py

# Define paths and parameters
image_dir = "/content/Character-Time-series-Matching/Vietnamese/Vietnamese_imgs"
output_annotations_file = "coco_annotations.json"
output_dir = "/content/blurred_images"
os.makedirs(output_dir, exist_ok=True)

# Load YOLOv8 model
yolov8_model = YOLO("/content/Character-Time-series-Matching/yolov8m.pt")

# Load second object detection model (object.pt)
object_model = Detection(
    size=(1280, 1280),
    weights_path="/content/Character-Time-series-Matching/Vietnamese/object.pt",
    device="cuda",  # Adjust device as needed
    iou_thres=0.4,
    conf_thres=0.1,
)

# Class mapping for combined models
class_mapping = {
    "rectangle license plate": "LicensePlate",
    "square license plate": "LicensePlate",
    "person": "person"
}
category_ids = {"person": 1, "LicensePlate": 2}

# Initialize COCO structure
coco_annotations = {
    "images": [],
    "annotations": [],
    "categories": [
        {"id": 1, "name": "person", "supercategory": "none"},
        {"id": 2, "name": "LicensePlate", "supercategory": "none"}
    ]
}

# Annotation ID tracker
annotation_id = 1

# Process images and collect detections
image_paths = sorted(glob.glob(os.path.join(image_dir, "frame_*.jpg")))

for image_id, image_path in enumerate(image_paths, start=1):
    print(f"Processing {image_path}...")
    # Open the image
    original_img = Image.open(image_path)
    img_cv2 = cv2.imread(image_path)
    height_orig, width_orig, _ = img_cv2.shape  # Original dimensions

    # Add image metadata to COCO annotations
    coco_annotations["images"].append({
        "id": image_id,
        "file_name": os.path.basename(image_path),
        "width": width_orig,
        "height": height_orig
    })

    # Run YOLOv8 prediction
    yolov8_results = yolov8_model.predict(source=image_path, conf=0.2, iou=0.5)
    yolov8_detections = yolov8_results[0]

    for detection in yolov8_detections.boxes:
        cls = int(detection.cls.item())  # Get class ID
        class_name = yolov8_model.names[cls]
        if class_name in class_mapping:
            mapped_name = class_mapping[class_name]
            category_id = category_ids[mapped_name]

            # Convert bounding box to COCO format
            x_min, y_min, x_max, y_max = map(float, detection.xyxy[0])
            bbox_width = x_max - x_min
            bbox_height = y_max - y_min
            area = bbox_width * bbox_height

            # Add annotation
            coco_annotations["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "bbox": [x_min, y_min, bbox_width, bbox_height],
                "area": area,
                "segmentation": [],  # Optional; empty for bounding boxes
                "iscrowd": 0
            })
            annotation_id += 1

    # Run object.pt model prediction
    h_resized, w_resized = 1280, 1280  # Assuming this is the resized dimension
    resized_img = ResizeImg(img_cv2, 1280)
    img_cv2, resized_img=preprocess_image(img_cv2)
    object_results, resized_img = object_model.detect(resized_img)

    for name, conf, box in object_results:
        if name in class_mapping:
            mapped_name = class_mapping[name]
            category_id = category_ids[mapped_name]

            # Map bounding box back to original dimensions
            x1, y1, x2, y2 = map(int, box)
            x1, y1 = map_resized_to_original(x1, y1, height_orig, width_orig, h_resized, w_resized)
            x2, y2 = map_resized_to_original(x2, y2, height_orig, width_orig, h_resized, w_resized)

            bbox_width = x2 - x1
            bbox_height = y2 - y1
            area = bbox_width * bbox_height

            # Add annotation
            coco_annotations["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "bbox": [x1, y1, bbox_width, bbox_height],
                "area": area,
                "segmentation": [],  # Optional; empty for bounding boxes
                "iscrowd": 0
            })
            annotation_id += 1

            # Crop license plate area for further processing (optional)
            license_plate_area = img_cv2[y1:y2, x1:x2]

# Save COCO annotations to JSON
with open(output_annotations_file, "w") as json_file:
    json.dump(coco_annotations, json_file, indent=4)

print(f"COCO annotations saved to {output_annotations_file}.")


Processing /content/Character-Time-series-Matching/Vietnamese/Vietnamese_imgs/frame_0000.jpg...

image 1/1 /content/Character-Time-series-Matching/Vietnamese/Vietnamese_imgs/frame_0000.jpg: 384x640 5 cars, 1 bus, 10.5ms
Speed: 2.5ms preprocess, 10.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)


NameError: name 'preprocess_image' is not defined

In [ ]:
from ultralytics import YOLO
import cv2
from PIL import Image, ImageFilter
import os
import glob

model = YOLO("yolov8m.pt")

image_dir = "/content/Character-Time-series-Matching/Vietnamese/Vietnamese_imgs"
output_dir = "/content/blurred_images"
os.makedirs(output_dir, exist_ok=True)

image_paths = sorted(glob.glob(os.path.join(image_dir, "frame_*.jpg")))

person_class_id = None
for class_id, class_name in model.names.items():
    if class_name == "person":
        person_class_id = class_id
        break

if person_class_id is None:
    print("Class 'person' not found in model names.")
else:
    for image_path in image_paths:
        print(f"Processing {image_path}...")
        original_img = Image.open(image_path)
        new_results = model.predict(source=image_path, conf=0.2, iou=0.5)
        new_result = new_results[0]

        for result in new_result.boxes:
            cls = result.cls
            if cls == person_class_id:
                x1, y1, x2, y2 = map(int, result.xyxy[0])
                person_area = original_img.crop((x1, y1, x2, y2))
                blurred_area = person_area.filter(ImageFilter.GaussianBlur(15))
                original_img.paste(blurred_area, (x1, y1))

        output_path = os.path.join(output_dir, os.path.basename(image_path))
        original_img.save(output_path)
        print(f"Blurred image saved at {output_path}")



image 1/1 /content/frame_3624.jpg: 384x640 4 persons, 15 cars, 1 motorcycle, 7 traffic lights, 11.2ms
Speed: 2.2ms preprocess, 11.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict5
Blurred image saved at /content/frame_3624_blurred.jpg


In [ ]:
%cd /content/Character-Time-series-Matching/Vietnamese
!python DETECTION.py --weights object.pt --imgsz 1280

/content/Character-Time-series-Matching/Vietnamese
/content/Character-Time-series-Matching/yolov5/models/experimental.py:96: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ck

In [ ]:
!cat /proc/cpuinfo
!free -h

processor	: 0
vendor_id	: AuthenticAMD
cpu family	: 23
model		: 49
model name	: AMD EPYC 7B12
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2249.998
cache size	: 512 KB
physical id	: 0
siblings	: 8
core id		: 0
cpu cores	: 4
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid tsc_known_freq pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy cr8_legacy abm sse4a misalignsse 3dnowprefetch osvw topoext ssbd ibrs ibpb stibp vmmcall fsgsbase tsc_adjust bmi1 avx2 smep bmi2 rdseed adx smap clflushopt clwb sha_ni xsaveopt xsavec xgetbv1 clzero xsaveerptr arat npt nrip_save umip rdpid
bugs		: sysret_ss_attrs null_seg spectre_v1 spectre_v2 spec_store_bypass retbleed smt_rsb srso
bogo

In [ ]:
!zip -r /content/out.zip /content/blurred_images

Streaming output truncated to the last 5000 lines.
  adding: content/blurred_images/frame_1404.jpg (deflated 1%)
  adding: content/blurred_images/frame_9768.jpg (deflated 2%)
  adding: content/blurred_images/frame_12362.jpg (deflated 2%)
  adding: content/blurred_images/frame_9926.jpg (deflated 2%)
  adding: content/blurred_images/frame_2064.jpg (deflated 1%)
  adding: content/blurred_images/frame_11751.jpg (deflated 1%)
  adding: content/blurred_images/frame_8857.jpg (deflated 2%)
  adding: content/blurred_images/frame_2364.jpg (deflated 2%)
  adding: content/blurred_images/frame_1621.jpg (deflated 1%)
  adding: content/blurred_images/frame_6434.jpg (deflated 2%)
  adding: content/blurred_images/frame_9169.jpg (deflated 2%)
  adding: content/blurred_images/frame_7816.jpg (deflated 2%)
  adding: content/blurred_images/frame_3935.jpg (deflated 3%)
  adding: content/blurred_images/frame_13509.jpg (deflated 1%)
  adding: content/blurred_images/frame_0162.jpg (deflated 1%)
  adding: conten

In [ ]:
import shutil
folder_path = '/content/Character-Time-series-Matching/Vietnamese/out'
shutil.rmtree(folder_path)

--------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from google.colab import files
files.download("/content/out.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd /content
!cp out.zip '/content/gdrive/MyDrive/'
!ls -lt '/content/gdrive/MyDrive/'

/content
total 24494999
-rw------- 1 root root 13556554848 Nov 29 02:12  out.zip
-rw------- 1 root root 11526313696 Oct 20 20:02  GX010085.MP4
lrw------- 1 root root          25 Oct  6 23:06 'My Drive' -> '/content/gdrive/My Drive/'
drwx------ 2 root root        4096 Sep 23 17:53  He_Cien
-rw------- 1 root root         175 Aug 10 03:23 'Untitled document.gdoc'
drwx------ 2 root root        4096 Aug  7 03:49 'Colab Notebooks'


In [ ]:
from ultralytics import YOLO  # required for loading the model

MODEL = YOLO("/content/last.pt")  #  loading the models
MODEL.predict("/content/frame_3624.jpg", save=True, show_labels=False)  # carrying out inference


In [ ]:
import json
from shapely.geometry import Polygon
from collections import defaultdict
import os


# Function to calculate IoU between two rectangles
def calculate_iou(box1, box2):
    poly1 = Polygon([(box1[0], box1[1]), (box1[0] + box1[2], box1[1]),
                     (box1[0] + box1[2], box1[1] + box1[3]), (box1[0], box1[1] + box1[3])])
    poly2 = Polygon([(box2[0], box2[1]), (box2[0] + box2[2], box2[1]),
                     (box2[0] + box2[2], box2[1] + box2[3]), (box2[0], box2[1] + box2[3])])

    if not poly1.is_valid or not poly2.is_valid:
        return 0.0

    intersection = poly1.intersection(poly2).area
    union = poly1.union(poly2).area
    return intersection / union if union > 0 else 0.0

# Function to parse COCO annotations
def parse_coco_annotations(coco_file):
    with open(coco_file, 'r') as f:
        coco_data = json.load(f)

    # Create a mapping of image_id to ground truth boxes
    gt_boxes = defaultdict(list)
    for ann in coco_data["annotations"]:
        image_id = ann["image_id"]
        bbox = ann["bbox"]  # COCO bbox format: [x, y, width, height]
        category_id = ann["category_id"]
        gt_boxes[image_id].append({"box": bbox, "category_id": category_id})

    # Create a mapping of image_id to image file names
    image_mapping = {img["id"]: img["file_name"] for img in coco_data["images"]}

    return gt_boxes, image_mapping

# Function to evaluate COCO-style annotations
def evaluate_coco(gt_file, model_results_file, iou_threshold=0.5):
    # Parse ground truth annotations
    gt_boxes, image_mapping = parse_coco_annotations(gt_file)

    # Load model results
    with open(model_results_file, 'r') as f:
        model_results_data = json.load(f)

    # Create a category mapping from model results to ground truth annotations
    category_mapping = {}
    for cat_gt in gt_boxes["categories"]:
        category_mapping[cat_gt["id"]] = cat_gt["name"]
    for cat_model in model_results_data["categories"]:
        category_mapping[cat_model["id"]] = cat_model["name"]

    # Map predictions by image_id
    model_results = defaultdict(list)
    for res in model_results_data.get("annotations", []):
        model_results[res["image_id"]].append(res)

    # Initialize evaluation results
    results = defaultdict(lambda: {"true_positive": 0, "false_positive": 0, "false_negative": 0})

    for image_id, image_name in image_mapping.items():
        # Get ground truth boxes for the current image
        gt_boxes_image = [gt["box"] for gt in gt_boxes.get(image_id, [])]

        # Get model predictions for the current image
        detected_boxes = [res['bbox'] for res in model_results.get(image_id, [])]
        matched = set()

        # Match ground truths with detections
        for gt_box in gt_boxes_image:
            best_iou = 0
            best_detection = None
            for i, det_box in enumerate(detected_boxes):
                iou = calculate_iou(gt_box, det_box)
                if iou > best_iou and i not in matched:
                    best_iou = iou
                    best_detection = i

            if best_iou >= iou_threshold and best_detection is not None:
                results[image_name]["true_positive"] += 1
                matched.add(best_detection)
            else:
                print(image_name)
                results[image_name]["false_negative"] += 1

        # Count unmatched detections as false positives
        results[image_name]["false_positive"] += len(detected_boxes) - len(matched)

    # Aggregate results
    total_tp = sum(r["true_positive"] for r in results.values())
    total_fp = sum(r["false_positive"] for r in results.values())
    total_fn = sum(r["false_negative"] for r in results.values())
    print(f"tp: {total_tp}")
    print(f"fp: {total_fp}")
    print(f"fn: {total_fn}")


    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1_score:.2f}")
    return results

# Example usage
gt_file = "/content/job8.json"
model_results_file = "/content/coco_annotations.json"

evaluate_coco(gt_file, model_results_file)

images/bellevue-GX010043-003483.jpg
images/bellevue-GX010043-007147.jpg
images/bellevue-GX010044-005465.jpg
images/bellevue-GX010044-007207.jpg
images/bellevue-GX010044-024985.jpg
tp: 138
fp: 207
fn: 5
Precision: 0.40
Recall: 0.97
F1 Score: 0.57


defaultdict(<function __main__.evaluate_coco.<locals>.<lambda>()>,
            {'images/bellevue-GX010043-000000.jpg': {'true_positive': 2,
              'false_positive': 1,
              'false_negative': 0},
             'images/bellevue-GX010043-000330.jpg': {'true_positive': 1,
              'false_positive': 3,
              'false_negative': 0},
             'images/bellevue-GX010043-001171.jpg': {'true_positive': 0,
              'false_positive': 4,
              'false_negative': 0},
             'images/bellevue-GX010043-001321.jpg': {'true_positive': 0,
              'false_positive': 1,
              'false_negative': 0},
             'images/bellevue-GX010043-001471.jpg': {'true_positive': 0,
              'false_positive': 1,
              'false_negative': 0},
             'images/bellevue-GX010043-001562.jpg': {'true_positive': 0,
              'false_positive': 0,
              'false_negative': 0},
             'images/bellevue-GX010043-001592.jpg': {'true_positive':